In [1]:
import cudf
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
df = cudf.read_parquet('../data/all_states_lat_long_age_sex.parquet/*')[['p_id','GISJOIN', 'sex', 'age']]

### filter dataset and keep people 25 and over

In [4]:
df = df.query('age >= 25 ')

In [5]:
df['GISJOIN_og'] = df.GISJOIN
df['GISJOIN'] = (df.GISJOIN/1000).astype('int')

In [6]:
df

,p_id,GISJOIN,sex,age,GISJOIN_og
0,35808,1000100208022,1,43,1000100208022016
1,35809,1000100208022,1,42,1000100208022016
2,35810,1000100208022,1,41,1000100208022016
3,35811,1000100208022,1,44,1000100208022016
4,35812,1000100208022,1,44,1000100208022016
...,...,...,...,...,...
308745304,308739899,56004509511002,1,33,56004509511002128
308745305,308739900,56004509511002,1,33,56004509511002128
308745306,308739901,56004509511002,1,34,56004509511002128
308745307,308739902,56004509511002,0,33,56004509511002136


In [7]:
df.dtypes

p_id          int32
GISJOIN       int64
sex            int8
age            int8
GISJOIN_og    int64
dtype: object

## Adding education block wise - Male & Female

<b>JN9E001</b>:     Total</br>
<b>JN9E002</b>:     Male</br>
<b>JN9E003</b>:     Male: No schooling completed</br>
<b>JN9E004</b>:     Male: Nursery to 4th grade</br>
<b>JN9E005</b>:     Male: 5th and 6th grade</br>
<b>JN9E006</b>:     Male: 7th and 8th grade</br>
<b>JN9E007</b>:     Male: 9th grade</br>
<b>JN9E008</b>:     Male: 10th grade</br>
<b>JN9E009</b>:     Male: 11th grade</br>
<b>JN9E010</b>:     Male: 12th grade, no diploma</br>
<b>JN9E011</b>:     Male: High school graduate, GED, or alternative</br>
<b>JN9E012</b>:     Male: Some college, less than 1 year</br>
<b>JN9E013</b>:     Male: Some college, 1 or more years, no degree</br>
<b>JN9E014</b>:     Male: Associate's degree</br>
<b>JN9E015</b>:     Male: Bachelor's degree</br>
<b>JN9E016</b>:     Male: Master's degree</br>
<b>JN9E017</b>:     Male: Professional school degree</br>
<b>JN9E018</b>:     Male: Doctorate degree</br>
<b>JN9E019</b>:     Female</br>
<b>JN9E020</b>:     Female: No schooling completed</br>
<b>JN9E021</b>:     Female: Nursery to 4th grade</br>
<b>JN9E022</b>:     Female: 5th and 6th grade</br>
<b>JN9E023</b>:     Female: 7th and 8th grade</br>
<b>JN9E024</b>:     Female: 9th grade</br>
<b>JN9E025</b>:     Female: 10th grade</br>
<b>JN9E026</b>:     Female: 11th grade</br>
<b>JN9E027</b>:     Female: 12th grade, no diploma</br>
<b>JN9E028</b>:     Female: High school graduate, GED, or alternative</br>
<b>JN9E029</b>:     Female: Some college, less than 1 year</br>
<b>JN9E030</b>:     Female: Some college, 1 or more years, no degree</br>
<b>JN9E031</b>:     Female: Associate's degree</br>
<b>JN9E032</b>:     Female: Bachelor's degree</br>
<b>JN9E033</b>:     Female: Master's degree</br>
<b>JN9E034</b>:     Female: Professional school degree</br>
<b>JN9E035</b>:     Female: Doctorate degree</br>

In [8]:
df_cat = cudf.read_csv('/home/ajay/data/census/all_states_block_group/nhgis0016_csv/nhgis0016_ds176_20105_2010_blck_grp.csv')

df_cat.STATEA = df_cat.STATEA.astype('int')
df_cat = df_cat.query('STATEA != 72')

df_cat.GISJOIN = df_cat.GISJOIN.str.replace('G', '').astype('int')

sex_by_education_cols = ['JN9E002','JN9E003','JN9E004','JN9E005','JN9E006','JN9E007','JN9E008','JN9E009','JN9E010','JN9E011','JN9E012','JN9E013','JN9E014','JN9E015','JN9E016','JN9E017','JN9E018','JN9E019','JN9E020','JN9E021','JN9E022','JN9E023','JN9E024','JN9E025','JN9E026','JN9E027','JN9E028','JN9E029','JN9E030','JN9E031','JN9E032','JN9E033','JN9E034','JN9E035']

sex_by_education_cols_male = ['JN9E002','JN9E003','JN9E004','JN9E005','JN9E006','JN9E007','JN9E008','JN9E009','JN9E010','JN9E011','JN9E012','JN9E013','JN9E014','JN9E015','JN9E016','JN9E017','JN9E018']
sex_by_education_cols_female = ['JN9E019','JN9E020','JN9E021','JN9E022','JN9E023','JN9E024','JN9E025','JN9E026','JN9E027','JN9E028','JN9E029','JN9E030','JN9E031','JN9E032','JN9E033','JN9E034','JN9E035']

sex_by_education_cols_without_total = ['JN9E003','JN9E004','JN9E005','JN9E006','JN9E007','JN9E008','JN9E009','JN9E010','JN9E011','JN9E012','JN9E013','JN9E014','JN9E015','JN9E016','JN9E017','JN9E018','JN9E020','JN9E021','JN9E022','JN9E023','JN9E024','JN9E025','JN9E026','JN9E027','JN9E028','JN9E029','JN9E030','JN9E031','JN9E032','JN9E033','JN9E034','JN9E035']
df_cat = df_cat[['GISJOIN']+sex_by_education_cols]

for i in df_cat.columns:
    if i != 'GISJOIN':
        df_cat[i] = df_cat[i].astype('int16')

In [9]:
df_cat

,GISJOIN,JN9E002,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,JN9E010,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
0,1000100201001,184,11,0,0,0,11,0,0,0,...,0,0,65,27,45,7,20,73,0,0
1,1000100201002,427,0,7,6,27,0,21,18,23,...,23,0,154,0,55,52,61,0,0,0
2,1000100202001,316,13,12,0,57,0,0,14,0,...,50,11,172,0,82,11,53,19,15,0
3,1000100202002,246,0,0,0,0,0,23,10,0,...,45,0,62,7,0,31,33,0,0,8
4,1000100203001,786,0,0,0,11,0,37,9,16,...,18,52,353,46,117,62,138,23,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,441,0,0,0,16,5,9,20,0,...,17,0,179,41,108,56,54,6,0,0
217736,56004509511002,741,0,0,0,29,14,9,26,0,...,32,10,212,45,74,62,60,45,0,0
217737,56004509513001,410,0,0,0,28,8,9,10,11,...,0,8,244,13,51,24,50,0,0,7
217738,56004509513002,414,0,0,0,0,0,31,13,17,...,10,0,128,85,63,17,99,2,0,0


In [10]:
all(df_cat[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values == df_cat.JN9E002.to_array())

True

In [11]:
all(df_cat[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values == df_cat.JN9E019.to_array())

True

In [12]:
for i in sex_by_education_cols_male:
    if i != 'JN9E002':
        df_cat[i] = df_cat[i]/df_cat['JN9E002'].astype('float32')#*100).floor().astype('int16')

for i in sex_by_education_cols_female:
    if i != 'JN9E019':
        df_cat[i] = df_cat[i]/df_cat['JN9E019'].astype('float32')#*100).floor().astype('int16')
        
df_cat.drop_column('JN9E002')
df_cat.drop_column('JN9E019')

In [13]:
df_cat

,GISJOIN,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,JN9E010,JN9E011,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
0,1000100201001,0.059783,0.000000,0.000000,0.000000,0.059783,0.000000,0.000000,0.000000,0.467391,...,0.000000,0.000000,0.262097,0.108871,0.181452,0.028226,0.080645,0.294355,0.000000,0.000000
1,1000100201002,0.000000,0.016393,0.014052,0.063232,0.000000,0.049180,0.042155,0.053864,0.306792,...,0.060052,0.000000,0.402089,0.000000,0.143603,0.135770,0.159269,0.000000,0.000000,0.000000
2,1000100202001,0.041139,0.037975,0.000000,0.180380,0.000000,0.000000,0.044304,0.000000,0.199367,...,0.107759,0.023707,0.370690,0.000000,0.176724,0.023707,0.114224,0.040948,0.032328,0.000000
3,1000100202002,0.000000,0.000000,0.000000,0.000000,0.000000,0.093496,0.040650,0.000000,0.126016,...,0.188285,0.000000,0.259414,0.029289,0.000000,0.129707,0.138075,0.000000,0.000000,0.033473
4,1000100203001,0.000000,0.000000,0.000000,0.013995,0.000000,0.047074,0.011450,0.020356,0.258270,...,0.019956,0.057650,0.391353,0.050998,0.129712,0.068736,0.152993,0.025499,0.007761,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,0.000000,0.000000,0.000000,0.036281,0.011338,0.020408,0.045351,0.000000,0.399093,...,0.036170,0.000000,0.380851,0.087234,0.229787,0.119149,0.114894,0.012766,0.000000,0.000000
217736,56004509511002,0.000000,0.000000,0.000000,0.039136,0.018893,0.012146,0.035088,0.000000,0.437247,...,0.057451,0.017953,0.380610,0.080790,0.132855,0.111311,0.107720,0.080790,0.000000,0.000000
217737,56004509513001,0.000000,0.000000,0.000000,0.068293,0.019512,0.021951,0.024390,0.026829,0.378049,...,0.000000,0.019560,0.596577,0.031785,0.124694,0.058680,0.122249,0.000000,0.000000,0.017115
217738,56004509513002,0.000000,0.000000,0.000000,0.000000,0.000000,0.074879,0.031401,0.041063,0.454106,...,0.023364,0.000000,0.299065,0.198598,0.147196,0.039720,0.231308,0.004673,0.000000,0.000000


### merge df_cat's index with df

In [14]:
df = df.merge(df_cat[['GISJOIN']], on='GISJOIN')

In [15]:
df

,p_id,GISJOIN,sex,age,GISJOIN_og
0,8719,1000100204002,1,53,1000100204002000
1,8720,1000100204002,1,52,1000100204002000
2,8721,1000100204002,1,52,1000100204002000
3,8722,1000100204002,1,50,1000100204002000
4,8723,1000100204002,1,50,1000100204002000
...,...,...,...,...,...
204127475,308730304,56004300002001,0,60,56004300002001528
204127476,308730305,56004300002001,0,60,56004300002001528
204127477,308730306,56004300002001,0,60,56004300002001528
204127478,308730307,56004300002001,0,61,56004300002001528


#### Make sure total ratio is a complete 1 (or 0 incase there are no values)

In [16]:
df1 = df.groupby(by=['GISJOIN', 'sex'], as_index=False).count()

In [17]:
df1 = df1.one_hot_encoding('sex', 'gender', [0,1])

In [18]:
df1.drop_column('sex')

In [19]:
df1.gender_0 = (df1.GISJOIN_og*df1.gender_0).astype('int')
df1.gender_1 = (df1.GISJOIN_og*df1.gender_1).astype('int')

In [20]:
df1.drop_column('GISJOIN_og')

In [21]:
df1 = df1.groupby('GISJOIN', as_index=False).sum()
df1

,GISJOIN,p_id,age,gender_0,gender_1
0,1000100201001,547,547,274,273
1,1000100201002,786,786,362,424
2,1000100202001,594,594,265,329
3,1000100202002,799,799,434,365
4,1000100203001,1531,1531,520,1011
...,...,...,...,...,...
216785,56004509511001,1065,1065,547,518
216786,56004509511002,1419,1419,838,581
216787,56004509513001,704,704,289,415
216788,56004509513002,984,984,468,516


#### Merge population in df with ratios in df_cat

In [22]:
df1 = df1.merge(df_cat, on='GISJOIN')

In [23]:
df1

,GISJOIN,p_id,age,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
0,5011900037123,1216,1216,600,616,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.029112,0.232897,0.171761,0.149927,0.000000,0.328967,0.087336,0.000000,0.000000
1,5011900037131,1307,1307,545,762,0.000000,0.000000,0.0,0.000000,0.004190,...,0.000000,0.000000,0.302358,0.073509,0.069348,0.099861,0.332871,0.081831,0.012483,0.024965
2,5011900037132,1252,1252,507,745,0.000000,0.043413,0.0,0.013473,0.000000,...,0.000000,0.000000,0.420063,0.054859,0.094044,0.026646,0.155172,0.108150,0.054859,0.000000
3,5011900038001,460,460,168,292,0.000000,0.000000,0.0,0.000000,0.000000,...,0.141869,0.000000,0.325260,0.027682,0.346021,0.000000,0.128028,0.031142,0.000000,0.000000
4,5011900038002,799,799,329,470,0.000000,0.000000,0.0,0.064286,0.114286,...,0.168317,0.113861,0.297030,0.133663,0.000000,0.113861,0.089109,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216785,55009509607002,1238,1238,676,562,0.005137,0.000000,0.0,0.003425,0.013699,...,0.027687,0.011401,0.403909,0.127036,0.133550,0.060261,0.140065,0.050489,0.009772,0.027687
216786,55009509607003,941,941,525,416,0.000000,0.000000,0.0,0.013100,0.000000,...,0.000000,0.021598,0.406048,0.038877,0.177106,0.151188,0.142549,0.045356,0.000000,0.000000
216787,55009509607004,794,794,368,426,0.000000,0.000000,0.0,0.050473,0.006309,...,0.034121,0.000000,0.496063,0.107612,0.047244,0.078740,0.104987,0.049869,0.007874,0.000000
216788,55009509608001,1868,1868,1028,840,0.000000,0.000000,0.0,0.000000,0.006110,...,0.012834,0.007487,0.380749,0.063102,0.239572,0.075936,0.142246,0.022460,0.006417,0.007487


In [24]:
#males
df1.to_pandas().query('JN9E003.isnull()')

,GISJOIN,p_id,age,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
80,12003100144101,72,72,0,72,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,6006509800041,9,9,0,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262,6003709800031,2,2,0,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1263,6003709800041,144,144,0,144,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267,6003709800131,59,59,0,59,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210904,55002500011021,8,8,8,0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
212857,51071009802001,5,5,0,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212858,51071009803001,1,1,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216497,54006100101012,121,121,0,121,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
#females
df1.to_pandas().query('JN9E020.isnull()')

,GISJOIN,p_id,age,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
80,12003100144101,72,72,0,72,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,6006509800041,9,9,0,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
660,1008100407002,53,53,52,1,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262,6003709800031,2,2,0,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1263,6003709800041,144,144,0,144,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212857,51071009802001,5,5,0,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212858,51071009803001,1,1,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213716,48020104121001,10,10,10,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214913,55000900211001,756,756,756,0,0.000000,0.000000,0.030989,0.098927,0.096544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df2 = df1.copy()

In [27]:
null_replace_equal_distribute_factor = (100/len(sex_by_education_cols_male[1:]))/100
null_replace_equal_distribute_factor

0.0625

In [28]:
#males
for i in sex_by_education_cols_male[1:]:
    df1[i] = ((df1[i] * df1.gender_0)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_0) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JN9E003.isnull()').index #everything is null for a row in this case

for i in sex_by_education_cols_male[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = null_replace_equal_distribute_factor

df1['diff_male'] = df1.gender_0.to_pandas() - df1[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1)

In [29]:
#females
# df2 = df1.copy()
for i in sex_by_education_cols_female[1:]:
    df1[i] = ((df1[i] * df1.gender_1)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_1) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JN9E020.isnull()').index #everything is null for a row in this case

for i in sex_by_education_cols_female[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = null_replace_equal_distribute_factor

df1['diff_female'] = df1.gender_1.to_pandas() - df1[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1)

In [30]:
df1

,GISJOIN,p_id,age,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,...,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035,diff_male,diff_female
0,5011900037123,1216,1216,600,616,0,0,0,0,0,...,143,105,92,0,202,53,0,0,4,4
1,5011900037131,1307,1307,545,762,0,0,0,0,2,...,230,56,52,76,253,62,9,19,7,3
2,5011900037132,1252,1252,507,745,0,22,0,6,0,...,312,40,70,19,115,80,40,0,3,7
3,5011900038001,460,460,168,292,0,0,0,0,0,...,94,8,101,0,37,9,0,0,2,2
4,5011900038002,799,799,329,470,0,0,0,21,37,...,139,62,0,53,41,0,0,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216785,55009509607002,1238,1238,676,562,3,0,0,2,9,...,226,71,75,33,78,28,5,15,6,7
216786,55009509607003,941,941,525,416,0,0,0,6,0,...,168,16,73,62,59,18,0,0,4,6
216787,55009509607004,794,794,368,426,0,0,0,18,2,...,211,45,20,33,44,21,3,0,5,5
216788,55009509608001,1868,1868,1028,840,0,0,0,0,6,...,319,53,201,63,119,18,5,6,6,6


In [31]:
df1.diff_male.max()

103

In [32]:
df2[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).unique()

array([3.99998546, 6.99999157, 2.99999855, ..., 7.99998973, 8.00000661,
       7.00000988])

In [33]:
print(df1.diff_male.sum() == (df1.gender_0.to_array().sum() - df1[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values.sum()))
print(df1.diff_female.sum() == (df1.gender_1.to_array().sum() - df1[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values.sum()))

True
True


In [34]:
print(df1.gender_0.to_array().sum() == df1[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df1.gender_1.to_array().sum() == df1[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

False
False


#### filling the difference

Since they don't add up to the total population, lets apply the largest remainder method, mentioned here -> https://revs.runtime-revolution.com/getting-100-with-rounded-percentages-273ffa70252b

In [35]:
import cupy

cp_array_male = cupy.array(df2[sex_by_education_cols_male[1:]].to_gpu_matrix())
cp_array_male = cupy.nan_to_num(cp_array_male/ cp_array_male.sum(axis=1, keepdims=True))
cp_array_male_sorted = cupy.asnumpy(cp_array_male.argsort())
cp_array_male = cupy.asnumpy(cp_array_male)

cp_array_female = cupy.array(df2[sex_by_education_cols_female[1:]].to_gpu_matrix())
cp_array_female = cupy.nan_to_num(cp_array_female/ cp_array_female.sum(axis=1, keepdims=True))
cp_array_female_sorted = cupy.asnumpy(cp_array_female.argsort())
cp_array_female = cupy.asnumpy(cp_array_female)

Handle remaining remainder values

In [36]:
from multiprocessing import  Pool
from functools import partial
import numpy as np
import pandas as pd

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [37]:
df1_pandas = df1.to_pandas()

count = 0

def applyrows(row):
    global count, cp_array_male_sorted, cp_array_male, cp_array_female_sorted, cp_array_female
    #males
    sorted_series_male = cp_array_male_sorted[row.name][::-1]
    i = row['diff_male']
    while i > 0:
        for sorted_index in sorted_series_male:
            sorted_val = cp_array_male[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JN9E0'+str(sorted_index+3).zfill(2)
            row[t_index] += t_value
            i -= t_value
    #females
    sorted_series_female = cp_array_female_sorted[row.name][::-1]
    i = row['diff_female']
    while i > 0:
        for sorted_index in sorted_series_female:
            sorted_val = cp_array_female[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JN9E0'+str(sorted_index+20).zfill(2)
            row[t_index] += t_value
            i -= t_value
    
    count += 1
    if count%10000 == 0: 
        print('                                  ', end='\r')
        print('count', count,row.name, end='\r')
    return row

# df2_pandas = df1_pandas.apply(applyrows, axis=1)
df2_pandas = parallelize_on_rows(df1_pandas, applyrows) 

In [38]:
df2_pandas

,GISJOIN,p_id,age,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,...,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035,diff_male,diff_female
0,5011900037123,1216,1216,600,616,0,0,0,0,0,...,143,105,92,0,202,53,0,0,4,4
1,5011900037131,1307,1307,545,762,0,0,0,0,2,...,230,56,55,76,253,62,9,19,7,3
2,5011900037132,1252,1252,507,745,0,22,0,9,0,...,319,40,70,19,115,80,40,0,3,7
3,5011900038001,460,460,168,292,0,0,0,0,0,...,96,8,101,0,37,9,0,0,2,2
4,5011900038002,799,799,329,470,0,0,0,21,37,...,139,62,0,53,41,0,0,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216785,55009509607002,1238,1238,676,562,3,0,0,2,9,...,233,71,75,33,78,28,5,15,6,7
216786,55009509607003,941,941,525,416,0,0,0,6,0,...,168,16,73,62,59,18,0,0,4,6
216787,55009509607004,794,794,368,426,0,0,0,18,2,...,211,50,20,33,44,21,3,0,5,5
216788,55009509608001,1868,1868,1028,840,0,0,0,0,6,...,319,53,201,63,119,18,5,6,6,6


In [39]:
print(df2_pandas.gender_0.values.sum() == df2_pandas[sex_by_education_cols_male[1:]].sum(axis=1).values.sum())
print(df2_pandas.gender_1.values.sum() == df2_pandas[sex_by_education_cols_female[1:]].sum(axis=1).values.sum())

True
True


In [40]:
df_final = cudf.from_pandas(df2_pandas)

In [41]:
df_final

,GISJOIN,p_id,age,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,...,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035,diff_male,diff_female
0,5011900037123,1216,1216,600,616,0,0,0,0,0,...,143,105,92,0,202,53,0,0,4,4
1,5011900037131,1307,1307,545,762,0,0,0,0,2,...,230,56,55,76,253,62,9,19,7,3
2,5011900037132,1252,1252,507,745,0,22,0,9,0,...,319,40,70,19,115,80,40,0,3,7
3,5011900038001,460,460,168,292,0,0,0,0,0,...,96,8,101,0,37,9,0,0,2,2
4,5011900038002,799,799,329,470,0,0,0,21,37,...,139,62,0,53,41,0,0,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216785,55009509607002,1238,1238,676,562,3,0,0,2,9,...,233,71,75,33,78,28,5,15,6,7
216786,55009509607003,941,941,525,416,0,0,0,6,0,...,168,16,73,62,59,18,0,0,4,6
216787,55009509607004,794,794,368,426,0,0,0,18,2,...,211,50,20,33,44,21,3,0,5,5
216788,55009509608001,1868,1868,1028,840,0,0,0,0,6,...,319,53,201,63,119,18,5,6,6,6


In [42]:
print(df_final.gender_0.to_array().sum() == df_final[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df_final.gender_1.to_array().sum() == df_final[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

True
True


In [43]:
df_final.drop_column('diff_male')
df_final.drop_column('diff_female')
# df_final.drop_column('GISJOIN')
# df_final.rename({'GISJOIN_og':'GISJOIN'}, inplace=True)

In [44]:
df_final.to_parquet('education_step_1_final.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


In [45]:
df.to_parquet('GISJOIN_to_GISJOIN_og_mapping.parquet')